# Exponential Moving Average (Long only)

We determine the 12-period exponential moving average (referred to as 'EMA12') and compare it with the price at that time. We (subjectively) select 12 since EMA12 would be the average price over one hour.

There are two conditions which we check.

If the price is greater than the EMA12, we go long. We continue to stay invested until the square-off condition is satisfied.

When the price becomes less than the EMA12, we square off our long position.

Our trading rules can be stated as

Buy when price > EMA12
Square off when price < EMA12

Import the required libraries

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings 

warnings.filterwarnings('ignore')

Fix the backtesting period

In [12]:
# Backtesting period should be last 55 days data in 5 minute level 
end = pd.datetime.now().date()
start = end - pd.Timedelta(days=55)

Create a class and create instance attributes/methods which are required for backtesting

In [13]:
# Create a class
class backtesting_EMA:
    
    # Create Instance attribute and methods
    def __init__(self,ticker,start,end,ema_range):
        
        self.ticker = ticker
        self.start = start
        self.end = end
        self.ema_range = ema_range
        
        self.fetch_data()
        self.indicators()
        self.signals()
        self.positions()
        self.returns()
        
    #To fetch the required data from yahoo finance    
    def fetch_data(self):
        self.df = yf.download(self.ticker,self.start,self.end,interval='5m')
    
    # To compute the SMA indicator
    def indicators(self):
        self.df['ema'] = self.df['Adj Close'].ewm(span=self.ema_range,adjust=False).mean()
    
    # To generate the long only signal
    def signals(self):
        self.df['signals'] = np.where(self.df['Adj Close'] > self.df['ema'],1,0)
    
    # To mark the positions taken
    def positions(self):
        self.df['positions'] = self.df['signals'].shift()
    
    #To compute the returns generated
    def returns(self):
        self.df['BnH Returns'] = self.df['Adj Close'].pct_change()
        self.df['Strategy Returns'] = self.df['positions'] * self.df['BnH Returns']
        # BnH returns is the returns generated if bought and hold it for the mentioned period simply without any strategy
        self.df['BnH Returns'] = 1+self.df['BnH Returns']
        # Strategy Returns is the returns generated due to implementation of this strategy on this historical data
        self.df['Strategy Returns'] = 1+self.df['Strategy Returns']
        Strat_Returns = self.df['Strategy Returns'].cumprod()[-1]
        BnH_Returns = self.df['BnH Returns'].cumprod()[-1]
        print('Buy and Hold Returns :', self.df['BnH Returns'].cumprod()[-1])
        print('Strategy Returns :', self.df['Strategy Returns'].cumprod()[-1])
        return Strat_Returns, BnH_Returns
    

In [14]:
# Try with Nifty index by passing EMA range as 12, this will return the buy and hold return & strategy return 
Nifty_backtesting_EMA = backtesting_EMA('^NSEI',start,end,12)

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.0000496582804133
Strategy Returns : 1.047818270507752


Optimise the strategy by considering few other indices with the various short term moving average range 

In [15]:
ema_list = []
Returns=[]
indices=[]
indice_list=['^NSEI','SPY','^CNXIT','^NSEBANK','^GSPC']
for i in range(8,15,1):
    for j in indice_list:
        a = backtesting_EMA(j,start,end,i)
        ema_list.append(i)
        Returns.append(a.returns())
        indices.append(j)
Strat = [x[0] for x in Returns]
BnH = [x[1] for x in Returns]
    

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.0000496582804133
Strategy Returns : 1.0502043514772885
Buy and Hold Returns : 1.0000496582804133
Strategy Returns : 1.0502043514772885
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.9534698226109998
Strategy Returns : 0.9990470269731443
Buy and Hold Returns : 0.9534698226109998
Strategy Returns : 0.9990470269731443
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.9391186837817751
Strategy Returns : 0.9997563494928184
Buy and Hold Returns : 0.9391186837817751
Strategy Returns : 0.9997563494928184
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 1.057767070269866
Strategy Returns : 1.0499262137819667
Buy and Hold Returns : 1.057767070269866
Strategy Returns : 1.0499262137819667
[*********************100%***********************]  1 of 1 completed
Buy and Hold Retu

In [16]:
# Create a dataframe and validate the returns generated for various indices with different EMA range
Results = pd.DataFrame({'Indices':indices,'EMA':ema_list,'Strategy Returns':Strat,'BnH Returns':BnH})


In [17]:
# Shows the record in descending order which has maximum return 
Results.sort_values(by='Strategy Returns',inplace=True,ascending=False)

Note : This is simple strategy to learn how to backtest and optimize the parameters by reusing the code.

In [18]:
Results.head(20)

,Indices,EMA,Strategy Returns,BnH Returns
33,^NSEBANK,14,1.068815,1.057767
18,^NSEBANK,11,1.064874,1.057767
23,^NSEBANK,12,1.063300,1.057767
13,^NSEBANK,10,1.062804,1.057767
28,^NSEBANK,13,1.059593,1.057767
8,^NSEBANK,9,1.059216,1.057767
15,^NSEI,11,1.054530,1.000050
30,^NSEI,14,1.052704,1.000050
0,^NSEI,8,1.050204,1.000050
3,^NSEBANK,8,1.049926,1.057767
